In [ ]:
import os
import sys
import tempfile
import subprocess as sub
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import Image, display

In [ ]:
def parse_graph(dot_file):
    treelet_count = 0
    raw = []
    
    with open(dot_file, "r") as f:
        for line in f:
            if '->' not in line:
                continue
                
            line = line.strip().split(" ")
            src, dst = int(line[0][1:]), int(line[2][1:])
            instance = len(line) > 3
            
            raw += [(src, dst, instance)]
            treelet_count = max(src, dst, treelet_count)
    
    treelet_count += 1
    instance_roots = set([x[1] for x in raw if x[2]])
    graph = {}
    
    for r in raw:
        if not r[2]:
            graph[r[0]] = graph.get(r[0], []) + [r[1]]
    
    treelets = set(range(0, treelet_count))
    
    main_graph = []
    stack = [0]
    
    while len(stack):
        n = stack.pop()
        for x in graph.get(n, []):
            main_graph += [(n, x)]
            stack.append(x)
            
    instance_graphs = {}
    
    for root in instance_roots:
        instance_graphs[root] = []
        
        stack = [root]
        while len(stack):
            n = stack.pop()
            
            for x in graph.get(n, []):
                instance_graphs[root] += [(n, x)]
                stack.append(x)
    
    return main_graph, instance_graphs

def read_csv(path):
    data = pd.read_csv(path)
    data['timestamp_s'] = (data.timestamp / 1000).astype(int)
    
    return (data.groupby(['timestamp_s', 'treeletId'])
                .agg({'raysDequeued': 'sum'}).reset_index())

def plot_graph(graph, data, timestamp):
    with tempfile.NamedTemporaryFile(suffix=".gv") as fin, \
         tempfile.NamedTemporaryFile(suffix=".jpg") as fout:
        fin.write(b'digraph bvh {\n')
        fin.write(b'  graph [K=0.6,size="9,9!",dpi=100,ratio="fill",scale=3];')
        fin.write(b'  node [shape=circle,style=filled'
                  b',fillcolor=white,label="",width=0.3];')
        fin.write(b'  edge [arrowhead=vee,arrowsize=0.5];')
        fin.write(b'  overlap = false;\n  splines = true;')
        for edge in graph:
            fin.write(f'  T{edge[0]} -> T{edge[1]};\n'.encode('utf-8'))
            
        for i in range(162):
            fin.write(f' T{i} [label="{i}"];\n'.encode('utf-8'))
            
        for index, row in data[(data.timestamp_s < timestamp) & (data.raysDequeued > 0)].iterrows():
            fin.write(f' T{int(row.treeletId)} [fillcolor="green"];\n'.encode('utf-8'))
            
        fin.write(b'}\n')
        fin.flush()
        
        sub.run(["sfdp", "-Tjpg", fin.name, "-o", fout.name])
        return Image(filename=fout.name)

In [ ]:
main_graph, instance_graphs = parse_graph("/data/r2t2/scenes/moana-full-nvidia/graph.dot")
treelet_data = read_csv("/data/r2t2/logs/2020-03-30/moana-full-1spp-2/treelets.csv")
plot_graph(main_graph, treelet_data, 30)